In [5]:
import sys
sys.path.append('/Users/dan/Code/Python/pg_scaffold/generated')

In [6]:

from app.core.db import SessionLocal

db = SessionLocal()
skip = 0
limit = 100

In [8]:
# Generated by pg-scaffolding 2025-08-17 11:13:30
from typing import List

from fastapi import APIRouter, Depends, HTTPException, Query
from sqlalchemy.orm import Session

from app.crud import user
from app.schemas import user as user_schema
from app.core.db import SessionLocal
from app.util import encrypt 

user_json = {"first_name":"dan",
            "last_name": "higgins",
            "email": "dphiggins3@gmail.com",
            "password": "Password"}

user_json["password"] = encrypt.hash_password(user_json["password"])
db_obj = None
try:
    user_in = user_schema.UserCreate.model_validate(user_json)
    db_obj = user.crud.create(db=db, obj_in=user_in)
except Exception as e:
        db.rollback()  # Rollback the transaction in case of error
        raise HTTPException(status_code=500, detail=str(e))

db_obj

In [9]:
from app.crud import user
from app.schemas import user as user_schema
from app.util import encrypt 

user_json = {"email": "dphiggins2@gmail.com",
            "password": "Password"}

errors = {}
# Lookup Email
filters = {"email":user_json["email"]}
db_obj = user.crud.filter_multi_like(db,filters)
user_objs = [user_schema.UserRead.model_validate(db_user) for db_user in db_obj]
# If we found an email check the password
if user_objs:
    user_obj = user_objs[0]
    valid_login = encrypt.verify_password(user_json["password"], user_obj.password)
    
    if not valid_login:
        errors["login"] = "Email or Password are incorrect."
        
    if not user_obj.is_verified:
        errors["is_verified"] = "Email has not been verified.\nCheck your email or reset password with Forgot Passowrd."
else:
     errors["login"] = "Email or Password are incorrect.."

if errors:
    print(errors)
else:
    print(user_objs[0])

{'is_verified': 'Email has not been verified.\nCheck your email or reset password with Forgot Passowrd.'}
